~~~python
from huggingface_hub import login
login(token="")
~~~

~~~python
from dotenv import load_dotenv
load_dotenv(dotenv_path="")
~~~

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
import torch
from diffusers import StableDiffusionPipeline

pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float32
).to("mps") # Usa "cpu", "cuda" para GPU o "mps" para Apple Silicon

prompt = "a futuristic city at sunset, ultra-detailed, 4K"
image = pipe(prompt).images[0]
#image.save("output_sb1-5.png") # Guarda la imagen generada
image

In [ ]:
pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float32
).to("mps")
prompt = "a futuristic city at sunset, ultra-detailed, 4K"
generator = torch.manual_seed(42)
image = pipe(prompt, generator=generator).images[0]
image

In [ ]:
pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float32
).to("mps")
prompt = "a futuristic city at sunset, ultra-detailed, 4K"
generator = torch.manual_seed(42)
image = pipe(prompt, generator=generator).images[0]
image

In [ ]:
import torch
from diffusers import StableDiffusion3Pipeline

pipe = StableDiffusion3Pipeline.from_pretrained("stabilityai/stable-diffusion-3-medium-diffusers", torch_dtype=torch.float16)
pipe = pipe.to("mps")

image = pipe(
    "A cat holding a sign that says hello world",
    negative_prompt="",
    num_inference_steps=28,
    guidance_scale=7.0,
    generator=torch.manual_seed(42)
).images[0]
image

In [ ]:
import torch
from diffusers import StableDiffusion3Pipeline

pipe = StableDiffusion3Pipeline.from_pretrained("stabilityai/stable-diffusion-3-medium-diffusers", torch_dtype=torch.float16)
pipe = pipe.to("mps")

image = pipe(
    "A cat holding a sign that says hello world",
    negative_prompt="",
    num_inference_steps=28,
    guidance_scale=7.0,
    generator=torch.manual_seed(42)
).images[0]
image

In [ ]:
import torch
from diffusers import StableDiffusion3Pipeline

pipe = StableDiffusion3Pipeline.from_pretrained("stabilityai/stable-diffusion-3-medium-diffusers", torch_dtype=torch.float16)
pipe.to("mps")

image = pipe(
    prompt="a photo of a cat holding a sign that says hello world",
    negative_prompt="blurry, low quality, text, dog",
    num_inference_steps=28,
    height=1024,
    width=1024,
    guidance_scale=7.0,
    generator=torch.manual_seed(42)
).images[0]
# image.save("sd3_hello_world.png")
image

In [ ]:
!pip install soundfile

In [ ]:
# Audio Generation

import torch
from huggingface_hub import login
from transformers import pipeline
from diffusers import DiffusionPipeline
from datasets import load_dataset
import soundfile as sf
from IPython.display import Audio

synthesiser = pipeline("text-to-speech", "microsoft/speecht5_tts", device='cpu')

embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
speaker_embedding = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)

speech = synthesiser("¡Hola a un ingeniero en inteligencia artificial, en camino hacia la maestría!",
                     forward_params={"speaker_embeddings": speaker_embedding})

sf.write("speech.wav", speech["audio"], samplerate=speech["sampling_rate"])
Audio("speech.wav")

In [ ]:
# Audio Generation

import torch
from transformers import pipeline
from datasets import load_dataset
import soundfile as sf
from IPython.display import Audio

synthesiser = pipeline("text-to-speech", "microsoft/speecht5_tts", device='cpu')

# Usar un embedding de voz del dataset actualizado
embeddings_dataset = load_dataset("anton-l/ecapa-tdnn-voxceleb-embeddings", split="train")
speaker_embedding = torch.tensor(embeddings_dataset[0]["embedding"]).unsqueeze(0)

speech = synthesiser(
    "¡Hola a un ingeniero en inteligencia artificial, en camino hacia la maestría!",
    forward_params={"speaker_embeddings": speaker_embedding}
)

sf.write("speech.wav", speech["audio"], samplerate=speech["sampling_rate"])
Audio("speech.wav")

In [ ]:
# Audio Generation

import torch
from transformers import pipeline
from datasets import load_dataset
import soundfile as sf
from IPython.display import Audio

synthesiser = pipeline("text-to-speech", "microsoft/speecht5_tts", device='cpu')
speech = synthesiser("¡Hola a un ingeniero en inteligencia artificial, en camino hacia la maestría!")
sf.write("speech.wav", speech["audio"], samplerate=speech["sampling_rate"])
Audio("speech.wav")

In [ ]:
!pip install fairseq sentencepiece


In [ ]:
from fairseq.checkpoint_utils import load_model_ensemble_and_task_from_hf_hub
from fairseq.models.text_to_speech.hub_interface import TTSHubInterface
import IPython.display as ipd


models, cfg, task = load_model_ensemble_and_task_from_hf_hub(
    "facebook/tts_transformer-es-css10",
    arg_overrides={"vocoder": "hifigan", "fp16": False}
)
model = models[0]
TTSHubInterface.update_cfg_with_data_cfg(cfg, task.data_cfg)
generator = task.build_generator([model], cfg)

text = "Hola, esta es una prueba."

sample = TTSHubInterface.get_model_input(task, text)
wav, rate = TTSHubInterface.get_prediction(task, model, generator, sample)

ipd.Audio(wav, rate=rate)

In [ ]:
from transformers import pipeline
import scipy

synthesiser = pipeline("text-to-speech", "suno/bark", device='cpu')

speech = synthesiser("Hello, my dog is cooler than you!", forward_params={"do_sample": True})

scipy.io.wavfile.write("bark_out.wav", rate=speech["sampling_rate"], data=speech["audio"])


In [ ]:
from transformers import pipeline
import scipy

synthesiser = pipeline("text-to-speech", "suno/bark", device='cpu')

speech = synthesiser("Hello, my dog is cooler than you!", forward_params={"do_sample": True})

# Convierte a float32 para evitar el error
audio = speech["audio"].astype("float32")
scipy.io.wavfile.write("bark_out.wav", rate=speech["sampling_rate"], data=audio)

In [ ]:

from kokoro import KPipeline
from IPython.display import display, Audio
import soundfile as sf
import torch
pipeline = KPipeline(lang_code='a')
text = '''
[Kokoro](/kˈOkəɹO/) is an open-weight TTS model with 82 million parameters. Despite its lightweight architecture, it delivers comparable quality to larger models while being significantly faster and more cost-efficient. With Apache-licensed weights, [Kokoro](/kˈOkəɹO/) can be deployed anywhere from production environments to personal projects.
'''
generator = pipeline(text, voice='af_heart')
for i, (gs, ps, audio) in enumerate(generator):
    print(i, gs, ps)
    display(Audio(data=audio, rate=24000, autoplay=i==0))
    sf.write(f'{i}.wav', audio, 24000)

In [ ]:

from kokoro import KPipeline
from IPython.display import display, Audio
import soundfile as sf
import torch
pipeline = KPipeline(lang_code='e')
text = '''
Cada vez que pienso en ti, mis ojos rompen en llanto; y muy triste me pregunto, ¿por qué te quiero tanto?
'''
generator = pipeline(text, voice='af_heart')
for i, (gs, ps, audio) in enumerate(generator):
    print(i, gs, ps)
    display(Audio(data=audio, rate=24000, autoplay=i==0))
    sf.write(f'{i}.wav', audio, 24000)